In [180]:
import requests
import pandas as pd
from typing import Text
from flask import Flask, render_template
import pandas as pd
import requests
from datetime import timedelta, datetime, date
import time


In [108]:
thisGw = 30
def dateAndTime(dataTime):
    return datetime.strptime(dateTime, "%Y-%m-%dT%H:%M:%SZ")

In [72]:
dateAndTime()

'2021-04-03T11:30:00Z'

In [176]:
#d = datetime.strptime('2021-04-03T11:30:00Z', "%Y-%m-%dT%H:%M:%SZ")
d = datetime.strptime('2021-04-03T11:30:00Z', "%Y-%m-%dT%H:%M:%SZ")
print(d)

2021-04-03 11:30:00


In [179]:
from_zone = tz.tzutc()
to_zone = tz.tzlocal()
d.strftime('%m-%d')
d.strftime('%d-%m-%H:%M')
d.replace(tzinfo=from_zone)
central = d.astimezone(to_zone)
central.strftime('%d-%m-%H:%M')




'03-04-11:30'

In [184]:
def getDateTime(utc_datetime):
    utc_datetime = datetime.strptime(utc_datetime, "%Y-%m-%dT%H:%M:%SZ")
    now_timestamp = time.time()
    offset = datetime.fromtimestamp(now_timestamp) - datetime.utcfromtimestamp(now_timestamp)
    return utc_datetime + offset

In [185]:
datetime_from_utc_to_local('2021-04-03T11:30:00Z')

datetime.datetime(2021, 4, 3, 13, 30)

In [61]:
def isGameInPlay(teamID):

    url = 'https://fantasy.premierleague.com/api/fixtures/?event='+ str(thisGw)
    fixtures = requests.get(url).json()
    for fix in fixtures:
        if teamID == fix['team_a'] or teamID == fix['team_h']:
            return fix['started'] and not fix['finished_provisional']

In [62]:
isGameInPlay(19)

False

In [35]:
test = not False

In [36]:
test

True